# 🟩 Masar Demand Modifier Notebook

This notebook defines and validates the logic for **computing demand modifiers** — numerical factors that adjust the baseline demand curve based on external influences.  
Instead of generating demand directly, this notebook produces a **multiplier** that reflects how much the passenger flow is expected to increase or decrease under different conditions.

---

### 🎯 Purpose
To create **scalable modifier functions** that adjust the baseline demand curve according to:
- Station attributes (capacity, type, and location)
- Temporal factors (weekday vs. weekend)
- Environmental factors (weather conditions)
- Scheduled events (public holidays, special occasions)

---

### ⚙️ Workflow
1️⃣ Load configuration and seed files (`calendar_events.csv`, `holidays.csv`)  
2️⃣ Define modifier logic per category (station, temporal, event, environment)  
3️⃣ Combine modifiers into one unified multiplier  
4️⃣ Export as `modifiers.csv` for use in occupancy generation notebooks

In [ ]:
!git clone https://github.com/Jana-Alrzoog/2025_GP_28.git
%cd 2025_GP_28


Cloning into '2025_GP_28'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 657 (delta 89), reused 1 (delta 1), pack-reused 485 (from 1)
Receiving objects: 100% (657/657), 4.57 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/2025_GP_28


In [ ]:
import yaml, json, csv
from datetime import datetime

seed_path = "/content/2025_GP_28/masar-sim/data/seeds"
config_path = "/content/2025_GP_28/masar-sim/sims/00_config.yaml"

with open(config_path) as f:
    config = yaml.safe_load(f)

seeds = {}
with open(f"{seed_path}/stations.json") as f:
    seeds["stations"] = json.load(f)
with open(f"{seed_path}/weather_patterns.json") as f:
    seeds["weather"] = json.load(f)
with open(f"{seed_path}/calendar_events.csv") as f:
    seeds["events"] = list(csv.DictReader(f))


print(" Loaded successfully")
print("Stations:", len(seeds["stations"]))
print("Events:", len(seeds["events"]))


 Loaded successfully
Stations: 6
Events: 21


Here is the main function that integrates all variables to produce the final modifier. It accepts a timestamp and station identifier as input, calculates the individual multipliers for each factor (weekend, weather, event, holiday), and then computes their product to capture the cumulative effect. The function returns a dictionary containing the final computed modifie

In [ ]:
import re
import pandas as pd

#Helper functions

def _match_station(sta, key):
    """Check if a given key matches station_id or code."""
    k = str(key).upper()
    return str(sta.get("station_id", "")).upper() == k or str(sta.get("code", "")).upper() == k

def _get_station(seeds, key):
    """Retrieve station record by ID or code."""
    for s in seeds["stations"]:
        if _match_station(s, key):
            return s
    raise ValueError(f"Station not found: {key}")

def _station_scale_from_capacity(stations, rec):
    """Compute station capacity scale relative to the network average."""
    caps = [r.get("capacity_platform") for r in stations if isinstance(r.get("capacity_platform"), (int, float))]
    mean_cap = (sum(caps) / len(caps)) if caps else 1500.0
    return float(rec.get("capacity_platform", mean_cap)) / float(mean_cap) if mean_cap > 0 else 1.0

def _iso(d):
    """Normalize date string to ISO format (YYYY-MM-DD) or return None."""
    try:
        return pd.to_datetime(d, errors="coerce").date().isoformat()
    except Exception:
        return None

# Main function
def compute_demand_modifier(ts, station_key, seeds, config):
    """Compute demand modifier based on date, station, weather, and events (no holidays)."""

    # Basic setup
    date_str = ts.date().isoformat()
    weekday  = ts.weekday()

    st = _get_station(seeds, station_key)
    station_scale = _station_scale_from_capacity(seeds["stations"], st)

    # Weekend adjustment Fri=4, Sat=5
    weekend_mult = float(config.get("multipliers", {}).get("weekend", 1.0)) if weekday in [4, 5] else 1.0

    # Weather adjustment
    weather_map = seeds.get("weather", {})
    w = weather_map.get(date_str)
    if w is None and isinstance(weather_map, dict):
        # Normalize weather dictionary keys
        norm_weather = { _iso(k): v for k, v in weather_map.items() }
        w = norm_weather.get(date_str, {"condition": "Sunny"})
    if w is None:
        w = {"condition": "Sunny"}
    weather_cond = w.get("condition", "Sunny")
    weather_mult = float(config.get("multipliers", {}).get("weather", {}).get(weather_cond, 1.0))

    # Event modifiers
    sid   = str(st.get("station_id", "")).upper()
    scode = str(st.get("code", "")).upper()
    event_mult = 1.0

    for ev in seeds.get("events", []):
        ev_date_iso = _iso(ev.get("date"))
        if ev_date_iso != date_str:
            continue

        # Collect all possible station identifiers and split multiple entries
        stations_field = (
            ev.get("stations_impacted") or ev.get("stations") or
            ev.get("station_id") or ev.get("station") or ev.get("station_code") or ""
        )
        tokens = [t.strip().upper() for t in re.split(r"[;,]", str(stations_field)) if t.strip()]

        # Match current station against any listed in the event
        if (sid and sid in tokens) or (scode and scode in tokens):
            et = ev.get("event_type", "Other")
            ev_mult = float(ev.get("demand_modifier", config.get("events", {}).get(et, 1.0)))
            event_mult = max(event_mult, ev_mult)

    # Final computation no holiday multiplier
    final = station_scale * weekend_mult * weather_mult * event_mult

    return {
        "station": scode or sid,
        "date": date_str,
        "weather": weather_cond,
        "station_scale": station_scale,
        "weekend_mult": weekend_mult,
        "weather_mult": weather_mult,
        "event_mult": event_mult,
        "final_demand_modifier": final
    }


In [ ]:
from datetime import datetime

# for example compute modifier for kafd station on national Day
ts = datetime(2025, 9, 23, 9, 0)
station_key = "KAFD"

result = compute_demand_modifier(ts, station_key, seeds, config)
result


{'station': 'KAFD',
 'date': '2025-09-23',
 'weather': 'Sunny',
 'station_scale': 1.5750000000000002,
 'weekend_mult': 1.0,
 'weather_mult': 1.0,
 'event_mult': 1.8,
 'final_demand_modifier': 2.8350000000000004}